# Running Tensorflow Models on Android for classifiying handwritten image

### About the project

This is a handwritten character image (MNIST) classifier that can run on any android device. The app stores a set of images (0-9) that we can cycle through and classify in order. It uses a pre-trained model to perform inference on the device. This idea can be applied to any images, both by using the camera and by pulling from the Web. We're using preloaded images so we can run the app in a simulator (no need for the device since it doesn't require a camera). 

### Installation Steps

If you already have Android Studio, skip to Step 5!

#### Step 1 Download Android studio

https://developer.android.com/studio/index.html

#### Step 2 Download Android SDK

`$ wget https://dl.google.com/android/android-sdk_r24.4.1-linux.tgz
$ tar xvzf android-sdk_r24.4.1-linux.tgz -C ~/tensorflow`

#### Step 3 Download SDK Build Tools

`$ cd ~/tensorflow/android-sdk-linux
$ tools/android update sdk --no-ui`

#### Step 4 Download Android NDK

`$ wget https://dl.google.com/android/repository/android-ndk-r12b-linux-x86_64.zip
$ unzip android-ndk-r12b-linux-x86_64.zip -d ~/tensorflow`

Set correct SDK and NDK versions in your workspace file

#### Step 5 Train Model in Python on your desktop or a server 

2 included options 

`tensorflow_model/mnist_convnet_keras.py
tensorflow_model/mnist_convnet.py `

Below we have the code both for traing and saving the model in keras and tensorflow

#### Step 6 Download the latest Tensorflow Android release package (AAR)

AAR (Android Archive) files are a convenient way to distribute packages– mainly libraries– for use with Android Studio and Gradle. Adding one of these files to your app requires creating some metadata files and updating your app’s gradle build files. Fortunately, Android Studio can handle all of this for us.

http://docs.onemobilesdk.aol.com/android-ad-sdk/adding-aar-files.html

### Step 7 Compile the code and run it on a simulator (or device if you have one)! 

In the digit recognizer app folder I have included all the files required for running it directly in your mobile.
Also included two pretrained model in the folder.

How I trined the model is given below.

https://github.com/kabyabasu/DigitRecognizer/tree/master/DigitRecognizer_APP


Here we will create a very basic regression model with softmax function to predict the hand written digit

Let’s set up our environment

In [7]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import numpy.ma as ma
import time
import math
import seaborn as sns

from PIL import Image, ImageOps
from sklearn.datasets import fetch_mldata

**Download and Extraction**

Here we downloaded the dataset from MNIST datset

In [ ]:
mnist_dataset = fetch_mldata('MNIST original')

**Exploration**

Here we explore the dataset and generate visualizations such as a histogram of Data Distribution for Each Class

In [9]:
mnist_dataset = fetch_mldata('MNIST original')
Y_0 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 0.)[0]]

In [14]:
print ('number of Zeros in MNIST dataset:', Y_0.shape[0])

number of Zeros in MNIST dataset: 6903


In [13]:
Y_1 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 1.)[0]]
Y_2 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 2.)[0]]
Y_3 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 3.)[0]]
Y_4 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 4.)[0]]
Y_5 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 5.)[0]]
Y_6 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 6.)[0]]
Y_7 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 7.)[0]]
Y_8 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 8.)[0]]
Y_9 = mnist_dataset['data'][np.where(mnist_dataset['target'] == 9.)[0]]

In [15]:
print ('number of Ones in MNIST dataset:', Y_1.shape[0])
print ('number of Twos in MNIST dataset:', Y_2.shape[0])
print ('number of Threes in MNIST dataset:', Y_3.shape[0])
print ('number of Fours in MNIST dataset:', Y_4.shape[0])
print ('number of Fives in MNIST dataset:', Y_5.shape[0])
print ('number of Sixs in MNIST dataset:', Y_6.shape[0])
print ('number of Sevens in MNIST dataset:', Y_8.shape[0])
print ('number of Eights in MNIST dataset:', Y_9.shape[0])

number of Oness in MNIST dataset: 7877
number of Twos in MNIST dataset: 6990
number of Threes in MNIST dataset: 7141
number of Fours in MNIST dataset: 6824
number of Fives in MNIST dataset: 6313
number of Sixs in MNIST dataset: 6876
number of Sevens in MNIST dataset: 6825
number of Eights in MNIST dataset: 6958


In [21]:
import plotly 
plotly.tools.set_credentials_file(username='kabyabasu', api_key='c0NKw9xhfO5sPxBAAYca')

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=['Zero', 'One', 'Two','Three', 'Four','Five','Six','Seven','Eight','Nine'],
            y=[6903, 7877, 6990, 7141, 6824, 6313, 6876, 6825, 6958]
    )]

py.iplot(data, filename='basic-bar')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kabyabasu/0 or inside your plot.ly account where it is named 'basic-bar'


***Model Training and Prediction***

Here we will train 3 model.

(A) Softmax regression Model

(B) Tensorflow Model

(C) Keras Model

(A) Softmax regression Model

Here we train a Softmax regression Model on MNIST data

In [38]:

# ==============================================================================

"""A very simple MNIST classifier.
See extensive documentation at
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

#from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def main(_):
  # Import data
  mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


  # Create the model
  x = tf.placeholder(tf.float32, [None, 784])
  W = tf.Variable(tf.zeros([784, 10]))
  b = tf.Variable(tf.zeros([10]))
  y = tf.matmul(x, W) + b

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])

  # The raw formulation of cross-entropy,
  #
  #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
  #                                 reduction_indices=[1]))
  #
  # can be numerically unstable.
  #
  # So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
  # outputs of 'y', and then average across the batch.
  cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
  train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

  sess = tf.InteractiveSession()
  tf.global_variables_initializer().run()
  # Train
  for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

  # Test trained model
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  test_accuracy = (sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))
    
  print('test accuracy %g' % test_accuracy)


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
test accuracy 0.9182


SystemExit: 

/home/kabya/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



The prediction accurecy of Softmax regression Model is ~92% [note the system error is due to the fact I am using ubuntu 14]

#### (B) SAVE A MODEL WITH TENSORFLOW

Here we will download the MNIST data set and with the help of tensorflow we will built a convolution
network and later we will save the model.



In [2]:
# Python 3.6.0
# tensorflow 1.1.0

import os
import os.path as path
import pandas as pd
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

from tensorflow.examples.tutorials.mnist import input_data

MODEL_NAME = 'mnist_convnet'
NUM_STEPS = 3000
BATCH_SIZE = 16

def model_input(input_node_name, keep_prob_node_name):
    x = tf.placeholder(tf.float32, shape=[None, 28*28], name=input_node_name)
    keep_prob = tf.placeholder(tf.float32, name=keep_prob_node_name)
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    return x, keep_prob, y_

def build_model(x, keep_prob, y_, output_node_name):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # 28*28*1

    conv1 = tf.layers.conv2d(x_image, 64, 3, 1, 'same', activation=tf.nn.relu)
    # 28*28*64
    pool1 = tf.layers.max_pooling2d(conv1, 2, 2, 'same')
    # 14*14*64

    conv2 = tf.layers.conv2d(pool1, 128, 3, 1, 'same', activation=tf.nn.relu)
    # 14*14*128
    pool2 = tf.layers.max_pooling2d(conv2, 2, 2, 'same')
    # 7*7*128

    conv3 = tf.layers.conv2d(pool2, 256, 3, 1, 'same', activation=tf.nn.relu)
    # 7*7*256
    pool3 = tf.layers.max_pooling2d(conv3, 2, 2, 'same')
    # 4*4*256

    flatten = tf.reshape(pool3, [-1, 4*4*256])
    fc = tf.layers.dense(flatten, 1024, activation=tf.nn.relu)
    dropout = tf.nn.dropout(fc, keep_prob)
    logits = tf.layers.dense(dropout, 10)
    outputs = tf.nn.softmax(logits, name=output_node_name)

    # loss
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))

    # train step
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # accuracy
    correct_prediction = tf.equal(tf.argmax(outputs, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary_op = tf.summary.merge_all()

    return train_step, loss, accuracy, merged_summary_op

def train(x, keep_prob, y_, train_step, loss, accuracy,
        merged_summary_op, saver):
    print("training start...")

    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    init_op = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init_op)

        tf.train.write_graph(sess.graph_def, 'out',
            MODEL_NAME + '.pbtxt', True)

        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter('logs/',
            graph=tf.get_default_graph())

        for step in range(NUM_STEPS):
            batch = mnist.train.next_batch(BATCH_SIZE)
            if step % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %f' % (step, train_accuracy))
            _, summary = sess.run([train_step, merged_summary_op],
                feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            summary_writer.add_summary(summary, step)

        saver.save(sess, 'out/' + MODEL_NAME + '.chkp')

        test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels,
                                    keep_prob: 1.0})
        print('test accuracy %g' % test_accuracy)

    print("training finished!")

def export_model(input_node_names, output_node_name):
    freeze_graph.freeze_graph('out/' + MODEL_NAME + '.pbtxt', None, False,
        'out/' + MODEL_NAME + '.chkp', output_node_name, "save/restore_all",
        "save/Const:0", 'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

def main():
    if not path.exists('out'):
        os.mkdir('out')

    input_node_name = 'input'
    keep_prob_node_name = 'keep_prob'
    output_node_name = 'output'

    x, keep_prob, y_ = model_input(input_node_name, keep_prob_node_name)

    train_step, loss, accuracy, merged_summary_op = build_model(x, keep_prob,
        y_, output_node_name)
    saver = tf.train.Saver()

    train(x, keep_prob, y_, train_step, loss, accuracy,
        merged_summary_op, saver)

    export_model([input_node_name, keep_prob_node_name], output_node_name)

if __name__ == '__main__':
    main()

training start...
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.125000
step 100, training accuracy 0.812500
step 200, training accuracy 0.875000
step 300, training accuracy 0.875000
step 400, training accuracy 0.875000
step 500, training accuracy 1.000000
step 600, training accuracy 1.000000
step 700, training accuracy 0.875000
step 800, training accuracy 0.937500
step 900, training accuracy 1.000000
step 1000, training accuracy 1.000000
step 1100, training accuracy 0.937500
step 1200, training accuracy 1.000000
step 1300, training accuracy 1.000000
step 1400, 

The prediction accurecy of Tensorflow Model is ~98%

#### SAVE A MODEL WITH KERAS

Here we will download the MNIST data set and with the help of keras we will built a convolution
network and later we will save the model.



In [8]:
# Python 3.6.0
# tensorflow 1.1.0
# Keras 2.0.4

import os
import os.path as path

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = 'mnist_convnet'
EPOCHS = 1
BATCH_SIZE = 128


def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test


def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 7*7*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model


def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, \
                  optimizer=keras.optimizers.Adadelta(), \
                  metrics=['accuracy'])

    model.fit(x_train, y_train, \
              batch_size=BATCH_SIZE, \
              epochs=EPOCHS, \
              verbose=1, \
              validation_data=(x_test, y_test))


def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


def main():
    if not path.exists('out'):
        os.mkdir('out')

    x_train, y_train, x_test, y_test = load_data()

    model = build_model()

    train(model, x_train, y_train, x_test, y_test)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")


if __name__ == '__main__':
    main()

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 187s - loss: 0.2334 - acc: 0.9242 - val_loss: 0.0464 - val_acc: 0.9859

The prediction accurecy of Keras Model is ~92% in testing and ~99% in validation data set